In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


Mounted at /content/drive


In [ ]:
dataset_path = "/content/drive/MyDrive/Dataset Model/archive/Dataset/Test"  # change to your folder path


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)


Found 8754 images belonging to 2 classes.
Found 2187 images belonging to 2 classes.


In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 2126s 8s/step - accuracy: 0.5898 - loss: 0.7072 - val_accuracy: 0.6392 - val_loss: 0.6315
Epoch 2/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 155s 565ms/step - accuracy: 0.6827 - loss: 0.5937 - val_accuracy: 0.6616 - val_loss: 0.6055
Epoch 3/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 154s 564ms/step - accuracy: 0.7084 - loss: 0.5625 - val_accuracy: 0.6845 - val_loss: 0.5883
Epoch 4/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 210s 595ms/step - accuracy: 0.7349 - loss: 0.5329 - val_accuracy: 0.6781 - val_loss: 0.5890
Epoch 5/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 153s 557ms/step - accuracy: 0.7258 - loss: 0.5366 - val_accuracy: 0.6763 - val_loss: 0.5905
Epoch 6/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 151s 553ms/step - accuracy: 0.7299 - loss: 0.5269 - val_accuracy: 0.6872 - val_loss: 0.5863
Epoch 7/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 152s 556ms/step - accuracy: 0.7392 - loss: 0.5178 - val_accuracy: 0.6950 - val_loss: 0.5731
Epoch 8/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 153s 558ms/step - accuracy: 0.7412 - l

In [ ]:
model.save("/content/drive/MyDrive/deepfake_classifier.h5")
print("Model Saved!")


Model Saved!


In [ ]:
!pip install onnx onnxruntime keras2onnx

import keras2onnx
import onnx
from tensorflow.keras.models import load_model

model = load_model("deepfake_model.h5")
onnx_model = keras2onnx.convert_keras(model, model.name)

onnx.save_model(onnx_model, "deepfake_model.onnx")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'onnx.mapping'

In [ ]:
!pip install tensorflowjs

!tensorflowjs_converter \
    --input_format keras \
    /content/drive/MyDrive/deepfake_classifier.h5 \
    /content/drive/MyDrive/tfjs_model/


2025-11-21 10:02:31.119649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763719351.140184   19867 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763719351.146236   19867 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763719351.161331   19867 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763719351.161357   19867 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763719351.161360   19867 computation_placer.cc:177] computation placer alr

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/MyDrive/test.jpg"  # change this
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("FAKE")
else:
    print("REAL")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/test.jpg'